In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./part.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_part_number = df_menu['Part_Number'].tolist()
list_brand = df_menu['Brand'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=' + str(i) + '&brand=' + str(j) for i, j in zip(list_part_number, list_brand)]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for part_number, url in zip(list_part_number, list_url):
    work.put_nowait((part_number, url))

def crawler():
    global df
    
    while not work.empty():
        part_number, url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                resp = requests.get(url, proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                info = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                src = info[0]['image_BG_Url']
                
                # = = = = = = = = = = = = = = =
                
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Number': part_number,
                                         'Src': src}])
                
                status = 'ok'

                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Number': part_number}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./src_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：438
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR124&brand=STD  <->  [ok] - 剩余数量：418
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR129&brand=STD  <->  [ok] - 剩余数量：417
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR120&brand=STD  <->  [ok] - 剩余数量：416
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR111&brand=STD  <->  [ok] - 剩余数量：415
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR101&brand=STD  <->  [ok] - 剩余数量：414
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR112&brand=STD  <->  [ok] - 剩余数量：413
https://ecatalog.smpcorp.com/V2/STD/ap

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR177&brand=STD  <->  [ok] - 剩余数量：366
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR168&brand=STD  <->  [ok] - 剩余数量：365
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR170&brand=STD  <->  [ok] - 剩余数量：364
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR189&brand=STI  <->  [ok] - 剩余数量：363
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR169&brand=STD  <->  [ok] - 剩余数量：362
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR188&brand=STI  <->  [ok] - 剩余数量：361
https://ecatalog.smpcorp.com/V2/STD/api/image/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR243&brand=STD  <->  [ok] - 剩余数量：313
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR268&brand=STD  <->  [ok] - 剩余数量：312
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR258&brand=STD  <->  [ok] - 剩余数量：311
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR274&brand=STD  <->  [ok] - 剩余数量：310
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR264&brand=STD  <->  [ok] - 剩余数量：309
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4202&brand=STD  <->  [ok] - 剩余数量：308
https://ecatalog.smpcorp.com/V2/STD/api/image

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4309&brand=STD  <->  [ok] - 剩余数量：260
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4313&brand=STD  <->  [ok] - 剩余数量：259
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4262&brand=STI  <->  [ok] - 剩余数量：258
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4294&brand=STI  <->  [ok] - 剩余数量：257
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4299&brand=STI  <->  [ok] - 剩余数量：256
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4316&brand=STI  <->  [ok] - 剩余数量：255
https://ecatalog.smpcorp.com/V2/STD/api/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4341&brand=STI  <->  [ok] - 剩余数量：207
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4236&brand=STD  <->  [ok] - 剩余数量：206
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR217&brand=STD  <->  [ok] - 剩余数量：205
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4354&brand=STI  <->  [ok] - 剩余数量：204
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4368&brand=STI  <->  [ok] - 剩余数量：203
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4366&brand=STI  <->  [ok] - 剩余数量：202
https://ecatalog.smpcorp.com/V2/STD/api/i

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4398&brand=STI  <->  [ok] - 剩余数量：155
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4405&brand=STD  <->  [ok] - 剩余数量：154
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4420&brand=STI  <->  [ok] - 剩余数量：153
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4429&brand=STI  <->  [ok] - 剩余数量：152
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4419&brand=STI  <->  [ok] - 剩余数量：151
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4408&brand=STD  <->  [ok] - 剩余数量：150
https://ecatalog.smpcorp.com/V2/STD/api/

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4480&brand=STI  <->  [error] - 剩余数量：104
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=F04014&brand=TEC  <->  [ok] - 剩余数量：103
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=F04018&brand=TEC  <->  [ok] - 剩余数量：102
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=F04013&brand=TEC  <->  [ok] - 剩余数量：101
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4491&brand=STI  <->  [error] - 剩余数量：100
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4445&brand=STI  <->  [error] - 剩余数量：99
https://ecatalog.smpcorp.com/V2/STD

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=ADR4507&brand=STI  <->  [error] - 剩余数量：51
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=G04005&brand=TEC  <->  [ok] - 剩余数量：50
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=F04103&brand=TEC  <->  [ok] - 剩余数量：49
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=F04099&brand=TEC  <->  [ok] - 剩余数量：48
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=G04017&brand=TEC  <->  [ok] - 剩余数量：47
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=F04025&brand=TEC  <->  [ok] - 剩余数量：46
https://ecatalog.smpcorp.com/V2/STD/api/image/ge

https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=J04044&brand=TEC  <->  [ok] - 剩余数量：0
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=J04047&brand=TEC  <->  [ok] - 剩余数量：0
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=J04035&brand=TEC  <->  [ok] - 剩余数量：0
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=J04062&brand=TEC  <->  [ok] - 剩余数量：0
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=J04002&brand=TEC  <->  [ok] - 剩余数量：0
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=J04060&brand=TEC  <->  [ok] - 剩余数量：0
https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
